In [1]:
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

In [3]:
batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307, ), (0.3081, ))
])

X_train = datasets.MNIST(root='../dataset/mnist/', train=True, download=False, transform=transform)
X_test = datasets.MNIST(root='../dataset/mnist/', train=False, download=False, transform=transform)
train_loader = DataLoader(X_train, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(X_test, shuffle=True, batch_size=batch_size)

In [11]:
import torch.nn.functional as F
class InceptionA(torch.nn.Module):
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        
        self.branch_pool = torch.nn.Conv2d(in_channels, 24, kernel_size=1)  
        
        self.branch1x1 = torch.nn.Conv2d(in_channels, 16, kernel_size=1)
        
        self.branch5x5_1 = torch.nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = torch.nn.Conv2d(16, 24, kernel_size=5, padding=2)
        
        self.branch3x3_1 = torch.nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3_2 = torch.nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3_3 = torch.nn.Conv2d(24, 24, kernel_size=3, padding=1)
    def forward(self, x):
        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)
        
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)
        branch3x3 = self.branch3x3_3(branch3x3)
        
        outputs = [branch_pool, branch1x1, branch5x5, branch5x5]
        return torch.cat(outputs, dim=1)

In [19]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(88, 20, kernel_size=5)
        
        self.incep1 = InceptionA(in_channels=10)
        self.incep2 = InceptionA(in_channels=20)
        
        self.mp = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(1408, 10)
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incep1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incep2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

In [20]:
model = Net()

In [21]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [22]:
def train(epoch):
    running_loss = 0.0
    for batch_index, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss
        if(batch_index % 300 == 299):
            print("[%d, %5d] loss: %.3f" % (epoch + 1, batch_index + 1, running_loss / 300))
            running_loss = 0.0

In [23]:
def test():
    correct = 0
    total = 0
    for data in test_loader:
        inputs, target = data
        outputs = model(inputs)
        
        _, predict = torch.max(outputs.data, dim=1)
        total += target.size(0)
        correct += (predict == target).sum().item()
        
    print("Accuracy on test set: %d %% [%d/%d]" % (100 * correct / total, correct, total))    

In [24]:
%%time
if __name__ == "__main__":
    for epoch in range(10):
        train(epoch)
        test()

[1,   300] loss: 0.799
[1,   600] loss: 0.180
[1,   900] loss: 0.122
Accuracy on test set: 97 % [9745/10000]
[2,   300] loss: 0.103
[2,   600] loss: 0.086
[2,   900] loss: 0.079
Accuracy on test set: 98 % [9808/10000]
[3,   300] loss: 0.069
[3,   600] loss: 0.067
[3,   900] loss: 0.067
Accuracy on test set: 98 % [9835/10000]
[4,   300] loss: 0.061
[4,   600] loss: 0.052
[4,   900] loss: 0.056
Accuracy on test set: 98 % [9862/10000]
[5,   300] loss: 0.049
[5,   600] loss: 0.049
[5,   900] loss: 0.046
Accuracy on test set: 98 % [9870/10000]
[6,   300] loss: 0.044
[6,   600] loss: 0.040
[6,   900] loss: 0.045
Accuracy on test set: 98 % [9878/10000]
[7,   300] loss: 0.041
[7,   600] loss: 0.040
[7,   900] loss: 0.035
Accuracy on test set: 98 % [9882/10000]
[8,   300] loss: 0.033
[8,   600] loss: 0.036
[8,   900] loss: 0.036
Accuracy on test set: 99 % [9903/10000]
[9,   300] loss: 0.033
[9,   600] loss: 0.032
[9,   900] loss: 0.033
Accuracy on test set: 98 % [9899/10000]
[10,   300] loss: 0

## Residual Net

In [25]:
import torch

In [28]:
class ResidualBlock(torch.nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.channels = channels
        
        self.conv1 = torch.nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(channels, channels, kernel_size=3, padding=1)
    
    def forward(self, x):   
        y = F.relu(self.conv1(x))
        y = self.conv2(y)
        return F.relu(x + y)

In [29]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 16, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(16, 32, kernel_size=5)
        
        self.mp = torch.nn.MaxPool2d(2)
        
        self.rblock1 = ResidualBlock(16)
        self.rblock2 = ResidualBlock(32)
        
        self.fc = torch.nn.Linear(512, 10)
        
    def forward(self, x):
        in_size = x.size(0)
        x = self.mp(F.relu(self.conv1(x)))
        x = self.rblock1(x)
        x = self.mp(F.relu(self.conv2(x)))
        x = self.rblock2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

In [30]:
model1 = Net()

In [31]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model1.parameters(), lr=0.01, momentum=0.5)

In [32]:
%%time
if __name__ == "__main__":
    for epoch in range(10):
        train(epoch)
        test()

[1,   300] loss: 0.026
[1,   600] loss: 0.025
[1,   900] loss: 0.031
Accuracy on test set: 98 % [9887/10000]
[2,   300] loss: 0.028
[2,   600] loss: 0.027
[2,   900] loss: 0.026
Accuracy on test set: 98 % [9887/10000]
[3,   300] loss: 0.027
[3,   600] loss: 0.027
[3,   900] loss: 0.026
Accuracy on test set: 98 % [9887/10000]
[4,   300] loss: 0.028
[4,   600] loss: 0.030
[4,   900] loss: 0.024
Accuracy on test set: 98 % [9887/10000]
[5,   300] loss: 0.028
[5,   600] loss: 0.028
[5,   900] loss: 0.025
Accuracy on test set: 98 % [9887/10000]
[6,   300] loss: 0.027
[6,   600] loss: 0.027
[6,   900] loss: 0.027
Accuracy on test set: 98 % [9887/10000]
[7,   300] loss: 0.029
[7,   600] loss: 0.026
[7,   900] loss: 0.027
Accuracy on test set: 98 % [9887/10000]
[8,   300] loss: 0.029
[8,   600] loss: 0.028
[8,   900] loss: 0.025
Accuracy on test set: 98 % [9887/10000]
[9,   300] loss: 0.029
[9,   600] loss: 0.026
[9,   900] loss: 0.027
Accuracy on test set: 98 % [9887/10000]
[10,   300] loss: 0